# Initialize Truveta SDK

In [1]:
# These are some commonly used R Packages.  
# The arrow package makes loading data with spark faster. 
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.notebook.study)

In [2]:
#Study specific library
library(MASS)
library(survival)
library(dplyr)
library(tidyr)
library(purrr)
library(gt)
library(sparklyr)
library(ggplot2)
library(reshape2)
library(gtsummary)

In [3]:
print("load snapshot")
con <- create_connection()
study <- get_study(con)
#print(study)
population_id = "ps-3ormi7swwukuhhu6kcqrqw4mue"
population <- get_population(con, study, title = "PancreaticMainPop")
snapshot <- get_latest_snapshot(con, population)
snapshot
# get list of tables from the snapshot
tables <- get_tables(con, snapshot)
tables

In [19]:
display_df(cases_control_allvar_new)

In [4]:
#Get your working directory
# use fs = true when reading and writing files locally

output_path_local <- get_output_path(con, study, fs = TRUE)
output_path_local

In [5]:
display_plot<- \(x, dpi = "screen", ...) {
    file <- tempfile()
    # dump as PNG
    ggplot2::ggsave(file, device = "png", plot = x, dpi = dpi, ...)
    # load as base64
    uri <- base64enc::dataURI(file = file, mime = "image/png")
    unlink(file)
    # display as HTML
    displayHTML(paste0('<img src="', uri, '">'))
}

In [6]:
t1 <- paste(output_path_local, "/cases_control_allvar_new.csv.r", sep = "")

# use read.csv to read file into a R dataframe
cases_control_allvar_new <- read.csv(t1)
#nrow(AdvChemoMed_tb)
##display_df(cases_control,10)

In [7]:
# Convert R DataFrame to Spark DataFrame
temp_data <- as.DataFrame(cases_control_allvar_new)
createOrReplaceTempView(temp_data, "data")

In [91]:
# Build path
file_to_write <- paste(output_path_local, "/cases_control_allvar_new.csv.r", sep = "")

# use write.csv to write your file
write.csv(cases_control_allvar_new, file_to_write, row.names = FALSE)

##### Install package

In [24]:
# Arrange the packages in the proper order of installation dependencies.
# Remove the optional 'quiet' parameter to obtain a detailed installation log.
packages <- c('car')
install_packages(con, packages, quiet=TRUE)

# Load the packages
library(car)

# Install a specific version of the package if multiple versions are available. The get_available_package function provides the package along with its version
#packages <- c('VGAM@1.1-10')
#install_packages(con, packages, quiet=TRUE)


In [ ]:
#Chi sq test to compare incidence of metastases for pancreatic cancer between different levels in each groups

# Create a binary variable for metastases
#data <- comorbidity
#data1 <- exposure
#data <- cases_control

# List of categorical variables to test
#categorical_vars <- c("Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus","BMI_Group")
#categorical_vars <- c("CKD", "T2DM", "Hepatitis", "CLD", "Hypertension", "OSA", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes", "Obesity_codes", "")

categorical_vars <- c("FOLFIRINOX1", "Gemc_Cis_Cap1", "Gemc_Mono1", "Chemo1", "RadiationFl1", "SurgeryFl1")

# Perform chi-square test for each variable
results <- lapply(categorical_vars, function(var) {

# Filter out "Unknown" levels
  filtered_data <- data1 %>%
    filter(.data[[var]] != "Unknown")

# Filter out groups with less than 6 records
  filtered_data <- filtered_data %>%
    group_by(!!sym(var)) %>%
    filter(n() >= 6) %>%
    ungroup()

# Create a contingency table
contingency_table <- table(filtered_data[[var]], filtered_data$indicator)
  
# Perform chi-square test
  test <- chisq.test(contingency_table)
  
 # Return results
  list(variable = var,
       contingency_table = contingency_table,
       chi_sq_stat = test$statistic,
       p_value = test$p.value,
       expected = test$expected)
})

# Print results
for (result in results) {
  cat("Variable:", result$variable, "\n")
  print(result$contingency_table)
  cat("Chi-sq statistic:", result$chi_sq_stat, "\n")
  cat("P-value:", result$p_value, "\n")
  cat("Expected frequencies:\n")
  print(result$expected)
  cat("\n=======================\n")
}


#### Variable defination

In [55]:
all <- c("AgeAtDiagnosis","Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "BMI_Group", "BMI","OthMetFLBase", "PanLocation", 
"CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia", "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes","Cancer",
"Gastroesophageal_refluxdisease", "abdominal_pain","Dyspnea","Anemia","FHOMND","comorbidScore", "RadiationFl1", "SurgeryFl1","AdvChemoMedFl1",
"DirectBilirubin","TotalBilirubin","ALP","Albumin","TotalBilirubin","AST","ALT","TotalProtein","ProthrombinT", "SerumLipase", "SerumPanAmylase")

#Comorbidity
comorbidity <- c("CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia", "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes","Cancer",
"Gastroesophageal_refluxdisease", "abdominal_pain","Dyspnea","Anemia","FHOMND")

#Exposure
Exposurea <- c("FluorouracilFl1","GemcitabineFl1","OxaliplatinFl1","LeucovorinFl1","IrinotecanFl1","CisplatinFl1","CapecitabineFl1","FOLFIRINOX1", "Gemc_Cis_Cap1", "Gemc_Mono1", "Chemo1", "RadiationFl1", "SurgeryFl1")
Exposureb <- c("FOLFIRINOX", "Gemc_Cis_Cap", "Gemc_Mono","RadiationFl1", "SurgeryFl1","AdvChemoMedFl1","AdvChemoMedFl2")

Exposure1 <- c("RadiationFl1", "SurgeryFl1","AdvChemoMedFl1","exposure")
Exposure2 <- c("notriptylinFl1","venlafaxineFl1", "metronidazoleFl1")

#Demographic and other
DemographicOth <- c("AgeAtDiagnosis","Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "Region", "BMI_Group", "BMI", "BMI_GROUP_impute","BMI_imputed","OthMetFLBase", "PanLocation")
TempVar <- c("Race_old", "BMI_Group", "BMI", "PanLocation_old")

#lab variables
lab_all <- c("DirectBilirubin","TotalBilirubin","ALP","Albumin","TotalBilirubin","AST","ALT","TotalProtein","ProthrombinT", "SerumLipase", "SerumPanAmylase")

# without lab factors
all_wo_lb <- c("AgeAtDiagnosis","Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "BMI_Group", "BMI","OthMetFLBase", "PanLocation", 
"CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia", "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes","Cancer",
"Gastroesophageal_refluxdisease", "abdominal_pain","Dyspnea","Anemia","FHOMND","comorbidScore", "RadiationFl1", "SurgeryFl1","AdvChemoMedFl1")

#### Univariate Analysis

In [56]:
# P-value formatting function
format_pval <- function(p) {
  if (p < 0.0001) "<0.0001"
  else if (p < 0.001) "<0.001"
  else if (p < 0.01) "<0.01"
  else if (p < 0.1) "<0.1"
  else " "
}

# Filter dataset
dataset <- cases_control_allvar_new 

# Define variable list
categorical_vars <- c(DemographicOth, comorbidity, Exposure2)

results <- lapply(categorical_vars, function(var) {
  model <- coxph(as.formula(paste("Surv(AVAL, indicator) ~", var)), data = dataset)
  coef_summary <- summary(model)$coefficients
  confint_summary <- summary(model)$conf.int

  data.frame(
    Variable = var,
    Level = rownames(coef_summary),
    `P-value` = sapply(coef_summary[, "Pr(>|z|)"], format_pval),
    `HR (95% CI)` = paste0(
      round(confint_summary[,"exp(coef)"], 3), " (",
      round(confint_summary[,"lower .95"], 3), ", ",
      round(confint_summary[,"upper .95"], 3), ")"
    ),
    stringsAsFactors = FALSE
  )
})


# Combine results into a single table
results_df <- as.data.frame(bind_rows(results))
#results_df <- bind_rows(results)

# Print results
display_df(results_df,100)
#print(results_df)

# Export if needed
# write.csv(results_df, "cox_univariable_summary.csv", row.names = FALSE)
### WHOLE data

In [57]:
# P-value formatting function
format_pval <- function(p) {
  if (p < 0.0001) "<0.0001"
  else if (p < 0.001) "<0.001"
  else if (p < 0.01) "<0.01"
  else if (p < 0.1) "<0.1"
  else " "
}

# Filter dataset
dataset <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

# Define variable list
categorical_vars <- c(DemographicOth, comorbidity, Exposure1, Exposure2)

results <- lapply(categorical_vars, function(var) {
  model <- coxph(as.formula(paste("Surv(AVAL, indicator) ~", var)), data = dataset)
  coef_summary <- summary(model)$coefficients
  confint_summary <- summary(model)$conf.int

  data.frame(
    Variable = var,
    Level = rownames(coef_summary),
    `P-value` = sapply(coef_summary[, "Pr(>|z|)"], format_pval),
    `HR (95% CI)` = paste0(
      round(confint_summary[,"exp(coef)"], 3), " (",
      round(confint_summary[,"lower .95"], 3), ", ",
      round(confint_summary[,"upper .95"], 3), ")"
    ),
    stringsAsFactors = FALSE
  )
})


# Combine results into a single table
results_df <- as.data.frame(bind_rows(results))
#results_df <- bind_rows(results)

# Print results
display_df(results_df,100)
#print(results_df)

# Export if needed
# write.csv(results_df, "cox_univariable_summary.csv", row.names = FALSE)

#### Remove Baseline LM

In [11]:
#library(glmnet)
#library(rms)

# Multivariate regression
#rm_bmi_na <- cases_control_allvar[!is.na(cases_control_allvar$BMI),]

#Exclude baseline LM
filter_data <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

dataset <- cases_control_allvar_new

#define intercept-only model
intercept_only <- coxph(Surv(AVAL,indicator)~1,data = dataset)

#define model with all predictors
# BMI not added
cox_full1 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + OthMetFLBase + PanLocation + BMI +
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND, data = dataset)

# Add the Exposure related variables
cox_full2 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + MaritalStatus + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND + RadiationFl1 + SurgeryFl1 + AdvChemoMedFl1, data = filter_data)

In [ ]:
#Multivariate Logistic  Regression
logistic_model <- glm(indicator ~ AgeAtDiagnosis + Ethnicity  + Sex + BMI_Group + OthMetFLBase + PanLocation + BMI + CKD + T2DM + Hepatitis + CLD + Hypertension + osa + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + AdvChemoMedFl1, data = pandata3,family = binomial(link = "logit"))

#Multivariate Cox  Regression
cox_model <- coxph(Surv(AVAL,indicator) ~ AgeAtDiagnosis + Ethnicity  + Sex + OthMetFLBase + 
CKD + T2DM + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + 
Cancer + abdominal_pain + Dyspnea + Anemia, data = cases_control_allvar)

#Stepwise Selection (Direction Both) - AIC based - Whole Data 
step_both_model <- step(intercept_only, direction='both', scope=formula(cox_full2), trace=0)

#perform backward elimination regression
step_backwrd_model <- step(cox_full1, direction = "backward", trace = 0)

#Prepare data for lasso and ridge

# Create survival object
y <- Surv(dataset$AVAL, dataset$indicator)

# Create matrix of predictors (drop intercept column)
X <- model.matrix(~ AgeAtDiagnosis + Ethnicity  + Sex + OthMetFLBase + 
CKD + T2DM + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer + abdominal_pain + Dyspnea + 
Anemia, data = dataset)[, -1]

# Lasso regression
# Lasso: alpha = 1
lasso_fit <- cv.glmnet(x = X, y = y, family = "cox", alpha = 1)

# Ridge Regression
# Ridge: alpha = 0
ridge_fit <- cv.glmnet(x = X, y = y, family = "cox", alpha = 0)

In [19]:
# copy paste whatever model need to be run
cox_model <- coxph(Surv(AVAL,indicator) ~  Ethnicity  + Sex + OthMetFLBase + abdominal_pain + AdvChemoMedFl1, data = filter_data)
summary(cox_model)

In [50]:
#Time -to-event analysis

dataset <- cases_control_allvar %>% filter(LivDtFl30 == "Y")

cox_full2 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + MaritalStatus + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND + RadiationFl1 + SurgeryFl1 + AdvChemoMedFl1, data = dataset)

step_backwrd_model <- step(cox_full2, direction = "backward", trace = 0)
summary(step_backwrd_model)


In [38]:
# Add Lab values column in the main table

all <- c("PersonId", "AVAL", "indicator","AgeAtDiagnosis","Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "BMI_Group", "BMI","OthMetFLBase", "PanLocation", 
"CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia", "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes","Cancer",
"Gastroesophageal_refluxdisease", "abdominal_pain","Dyspnea","Anemia","FHOMND","comorbidScore", "RadiationFl1", "SurgeryFl1","AdvChemoMedFl1",
"DirectBilirubin","TotalBilirubin","ALP","Albumin","TotalBilirubin","AST","ALT","TotalProtein")

temp <- cases_control_allvar %>% dplyr::select(-AVAL,-indicator) %>% left_join(lab_test_result, by = "PersonId")

temp <- temp[,all]

df_lab <- temp[complete.cases(temp),]

cox_full_temp <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + MaritalStatus + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND + RadiationFl1 + SurgeryFl1 + AdvChemoMedFl1 + DirectBilirubin + TotalBilirubin + Albumin + 
TotalProtein + AST + ALT + ALP, data = df_lab)

step_backwrd_model <- step(cox_full_temp, direction = "backward", trace = 0)
summary(step_backwrd_model)

### STRATIFIED ANALYSIS

In [96]:
filter_data <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

othmet_data1 <- filter_data[filter_data$OthMetFLBase == "Y",]
othmet_data0 <- filter_data[filter_data$OthMetFLBase == "N",]

#define intercept-only model
intercept_only1 <- coxph(Surv(AVAL,indicator)~1,data = othmet_data1)
intercept_only0 <- coxph(Surv(AVAL,indicator)~1,data = othmet_data0)

cox_full_othmet1 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + PanLocation + BMI_imputed +
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND, data = othmet_data1)

cox_full_othmet0 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + PanLocation + BMI_imputed +
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND, data = othmet_data0)

#Stepwise Selection (Direction Both) - AIC based - Whole Data 
step_both_model1 <- step(intercept_only1, direction='both', scope=formula(cox_full_othmet1), trace=0)
step_both_model0 <- step(intercept_only0, direction='both', scope=formula(cox_full_othmet0), trace=0)

cat(nrow(othmet_data1),":",nrow(othmet_data0))

In [7]:
filter_data <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

surgery_data1 <- filter_data[filter_data$SurgeryFl1 == 1,]
surgery_data0 <- filter_data[filter_data$SurgeryFl1 == 0,]

#define intercept-only model
intercept_only1 <- coxph(Surv(AVAL,indicator)~1,data = surgery_data1)
intercept_only0 <- coxph(Surv(AVAL,indicator)~1,data = surgery_data0)

cox_full_surgery1 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND + RadiationFl1 + AdvChemoMedFl1 + exposure, data = surgery_data1)

cox_full_surgery0 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND + RadiationFl1 + AdvChemoMedFl1 + exposure, data = surgery_data0)

#Stepwise Selection (Direction Both) - AIC based - Whole Data 
step_both_model1 <- step(intercept_only1, direction='both', scope=formula(cox_full_surgery1), trace=0)
step_both_model0 <- step(intercept_only0, direction='both', scope=formula(cox_full_surgery0), trace=0)

cat(nrow(surgery_data1),":",nrow(surgery_data0))

In [10]:
summary(step_both_model1)

In [8]:
summary(step_both_model0)

In [111]:
# Extract summary from stepwise models
summary1 <- summary(step_both_model1)
summary0 <- summary(step_both_model0)

# Convert to data frames
df1 <- data.frame(
  Variable = rownames(summary1$coefficients),
  HR_Y = round(summary1$coefficients[, "exp(coef)"], 2),
  LCI_Y = round(summary1$conf.int[, "lower .95"], 2),
  UCI_Y = round(summary1$conf.int[, "upper .95"], 2),
  P_value_Y = round(summary1$coefficients[, "Pr(>|z|)"], 3)
  #Group = "OthMetFLBase = Y"
)

df0 <- data.frame(
  Variable = rownames(summary0$coefficients),
  HR_N = round(summary0$coefficients[, "exp(coef)"], 2),
  LCI_N = round(summary0$conf.int[, "lower .95"], 2),
  UCI_N = round(summary0$conf.int[, "upper .95"], 2),
  P_value_N = round(summary0$coefficients[, "Pr(>|z|)"], 3)
 # Group = "OthMetFLBase = N"
)

# Combine both for comparison
combined_results_surgery <- merge(df1,df0,by="Variable")

# View combined results
#display_df(combined_results_othmet)


In [112]:
display_df(combined_results_surgery,50)

### Check Assumptions

In [93]:
dataset <- cases_control_allvar_new

#define intercept-only model
intercept_only <- coxph(Surv(AVAL,indicator)~1,data = dataset)

cox_full1 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + OthMetFLBase + PanLocation + BMI_imputed +
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND, data = dataset)

#Stepwise Selection (Direction Both) - AIC based - Whole Data 
step_both_model <- step(intercept_only, direction='both', scope=formula(cox_full1), trace=0)

test.ph <- cox.zph(step_both_model)
test.ph


In [95]:
library("survival")
library("survminer")
ggcoxzph(test.ph)

### Create Tables

In [67]:
#Table 1 : Characteristics for Whole data

#dataset <- cases_control_allvar_new

#Table 2 : Characteristics for data with LM at baseline
dataset <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

In [50]:
count_wo_baselm <- rbind(df_out,summary_cat)
summary_wo_baselm <- results_df %>% left_join(count_wo_baselm,by = c("Variable","Level"))
display_df(summary_wo_baselm)

In [19]:
# P-value formatting function
format_pval <- function(p) {
  if (p < 0.0001) "<0.0001"
  else if (p < 0.001) "<0.001"
  else if (p < 0.01) "<0.01"
  else if (p < 0.1) "<0.1"
  else " "
}

# Variables
vars_cat <- c(comorbidity,DemographicOth)

# Table 1: Categorical/Binary + AgeAtDiagnosis
summary_cat <- map_dfr(vars_cat, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

  if (var == "AgeAtDiagnosis") {
    median_case <- round(median(df$AgeAtDiagnosis[df$indicator == 1], na.rm = TRUE), 1)
    median_control <- round(median(df$AgeAtDiagnosis[df$indicator == 0], na.rm = TRUE), 1)
    IQR_case <- round(IQR(df$AgeAtDiagnosis[df$indicator == 1], na.rm = TRUE),1)
    IQR_control <- round(IQR(df$AgeAtDiagnosis[df$indicator == 0], na.rm = TRUE),1)

    model <- coxph(Surv(AVAL, indicator) ~ AgeAtDiagnosis, data = df)
    s <- summary(model)
    HR <- round(s$coef[2], 2)
    LCI <- round(s$conf.int[,"lower .95"], 3)
    UCI <- round(s$conf.int[,"upper .95"], 3)
    p_val <- format_pval(s$coef[5])

    dataframe <- data.frame(
      `Variable (Level)` = "AgeAtDiagnosis:median(IQR)",
      `Cases` = paste0(median_case," (",IQR_case,")"),
      `Controls` = paste0(median_control," (",IQR_control,")"),
      `Total (n, %)` = paste0(nrow(df), " (", round(nrow(df)/nrow(dataset)*100,1), "%)"),
      `HR (95% CI)` = paste0(HR, " (", LCI, ", ", UCI, ")"),
      `P-value` = p_val
    )
    return(dataframe)
  }
 else {
  counts <- df %>%
    group_by(group = .data[[var]], indicator) %>%
    summarise(n = n(), .groups = "drop") %>%
    pivot_wider(names_from = indicator, values_from = n, values_fill = 0, names_prefix = "Group_") %>%
    mutate(
      Total = Group_0 + Group_1,
      Pct = round(Total / sum(Total) * 100, 1),
      `Variable (Level)` = paste0(var, ": ", group),
      `Cases` = Group_1,
      `Controls` = Group_0,
      `Total (n, %)` = paste0(Total, " (", Pct, "%)")
    )

  model <- tryCatch(coxph(Surv(AVAL, indicator) ~ .data[[var]], data = df), error = function(e) NULL)

  if (!is.null(model)) {
    s <- summary(model)
    hr_df <- data.frame(
      Level = rownames(s$coef),
      HR = round(s$coef[, 2], 2),
      LCI = round(s$conf.int[,"lower .95"], 2),
      UCI = round(s$conf.int[,"upper .95"], 2),
      Pval = s$coef[, 5]
    ) %>%
      mutate(`HR (95% CI)` = paste0(HR, " (", LCI, ", ", UCI, ")"),
             `P-value` = format_pval(Pval)) %>%
      select(Level, `HR (95% CI)`, `P-value`)

    counts <- counts %>%
      left_join(hr_df, by = c("group" = "Level")) %>%
      mutate(
        `HR (95% CI)` = ifelse(is.na(`HR (95% CI)`), "Ref", `HR (95% CI)`),
        `P-value` = ifelse(is.na(`P-value`), "Ref", `P-value`)
      )
  } else {
    counts <- counts %>% mutate(`HR (95% CI)` = NA, `P-value` = NA)
  }

counts %>%
  arrange(match(group, ref_level)) %>%
  select(all_of(c("Variable (Level)", "Cases", "Controls", "Total (n, %)", "HR (95% CI)", "P-value")))

})

#### Table for Continuous variable

In [75]:
vars_cont <- c("AgeAtDiagnosis","BMI_imputed")

summary_cont <- map_dfr(vars_cont, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

  median_case <- round(median(df[[var]][df$indicator == 1], na.rm = TRUE), 1)
  median_control <- round(median(df[[var]][df$indicator == 0], na.rm = TRUE), 1)
  IQR_case <- round(IQR(df[[var]][df$indicator == 1], na.rm = TRUE), 1)
  IQR_control <- round(IQR(df[[var]][df$indicator == 0], na.rm = TRUE), 1)

  model <- coxph(Surv(AVAL, indicator) ~ .data[[var]], data = df)
  s <- summary(model)
  HR <- round(s$coef[2], 2)
  LCI <- round(s$conf.int[,"lower .95"], 3)
  UCI <- round(s$conf.int[,"upper .95"], 3)
  p_val <- format_pval(s$coef[5])

 return(data.frame(
    `Variable (Level)` = paste0(var, ": median (IQR)"),
    `Cases (IQR)` = paste0(median_case, " (", IQR_case, ")"),
    `Controls (IQR)` = paste0(median_control, " (", IQR_control, ")"),
    `Total` = nrow(df),
    `HR(95% CI)` = paste0(HR, " (", LCI, ", ", UCI, ")"),
    `P-value` = p_val
  ))

})

display_df(summary_cont)

In [69]:
# AgeAtDiagnosis
df <- dataset
median_case <- round(median(df$AgeAtDiagnosis[df$indicator == 1], na.rm = TRUE), 1)
median_control <- round(median(df$AgeAtDiagnosis[df$indicator == 0], na.rm = TRUE), 1)
IQR_case <- round(IQR(df$AgeAtDiagnosis[df$indicator == 1], na.rm = TRUE),1)
IQR_control <- round(IQR(df$AgeAtDiagnosis[df$indicator == 0], na.rm = TRUE),1)

model1 <- coxph(Surv(AVAL, indicator) ~ AgeAtDiagnosis, data = df)
s1 <- summary(model1)
HR1 <- round(s1$coef[,"exp(coef)"], 2)
LCI1 <- round(s1$conf.int[,"lower .95"], 3)
UCI1 <- round(s1$conf.int[,"upper .95"], 3)
p_val1 <- format_pval(s1$coef[,"Pr(>|z|)"])

dataframe1 <- data.frame(
  `Variable` = "AgeAtDiagnosis",
  `Level` = "AgeAtDiagnosis",
  `Cases` = paste0(median_case," (",IQR_case,")"),
  `Controls` = paste0(median_control," (",IQR_control,")"),
  `HR (95% CI)` = paste0(HR1, " (", LCI1, ", ", UCI1, ")"),
  `P-value` = p_val1
)

#dataframe1 <- dataframe1 %>%mutate(`Total (%)` = "6068 (100%)") %>% dplyr::select(`Variable`, Level,Cases, Controls, `Total (%)`,` HR (95% CI)`,P-value)

# BMI
median_case <- round(median(df$BMI_imputed[df$indicator == 1], na.rm = TRUE), 1)
median_control <- round(median(df$BMI_imputed[df$indicator == 0], na.rm = TRUE), 1)
IQR_case <- round(IQR(df$BMI_imputed[df$indicator == 1], na.rm = TRUE),1)
IQR_control <- round(IQR(df$BMI_imputed[df$indicator == 0], na.rm = TRUE),1)

model2 <- coxph(Surv(AVAL, indicator) ~ BMI_imputed, data = df)
s2 <- summary(model2)
HR2 <- round(s2$coef[,"exp(coef)"], 2)
LCI2 <- round(s2$conf.int[,"lower .95"], 3)
UCI2 <- round(s2$conf.int[,"upper .95"], 3)
p_val2 <- format_pval(s2$coef[,"Pr(>|z|)"])

dataframe2 <- data.frame(
  `Variable` = "BMI_imputed",
  `Level` = "BMI_imputed",
  `Cases` = paste0(median_case," (",IQR_case,")"),
  `Controls` = paste0(median_control," (",IQR_control,")"),
 `HR (95% CI)` = paste0(HR2, " (", LCI2, ", ", UCI2, ")"),
 `P-value` = p_val2
)

#dataframe2 <- dataframe2 %>% mutate(`Total (%)` = "6068 (100%)") %>% dplyr::select(`Variable`, Level,Cases, Controls, `Total (%)`,` HR (95% CI)`,P-value)

# Combine
df_out <- rbind(dataframe1, dataframe2)
display_df(df_out)  # or View(df_out) in RStudio


In [46]:
nrow(dataset)

In [ ]:
summary_cat <- map_dfr(vars_cat, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

  counts <- df %>%
    group_by(group = .data[[var]], indicator) %>%
    summarise(n = n(), .groups = "drop") %>%
    pivot_wider(names_from = indicator, values_from = n, values_fill = 0, names_prefix = "Group_") %>%
    mutate(
      group = as.character(group),
      Total = Group_0 + Group_1,
      Pct = round(Total / sum(Total) * 100, 1),
      `Variable (Level)` = paste0(var, ": ", group),
      `Cases` = Group_1,
      `Controls` = Group_0,
      `Total (%)` = paste0(Total, " (", Pct, "%)"),
      Var_to_merge = paste0(var,group)
    )

  model <- coxph(Surv(AVAL, indicator) ~ .data[[var]], data = df)

    s <- summary(model)
    hr_df <- data.frame(
      Var_to_merge = rownames(s$coef),
      HR = round(s$coef[, 2], 2),
      LCI = round(s$conf.int[,"lower .95"], 2),
      UCI = round(s$conf.int[,"upper .95"], 2),
      Pval = s$coef[, 5]
    ) %>%
      mutate(`HR (95% CI)` = paste0(HR, " (", LCI, ", ", UCI, ")"),
             `P-value` = format_pval(Pval)) %>%
      select(Level, `HR (95% CI)`, `P-value`)

    counts <- counts %>%
      left_join(hr_df, by = "Var_to_merge") %>%
      mutate(
        `HR (95% CI)` = ifelse(is.na(`HR (95% CI)`), "Ref", `HR (95% CI)`),
        `P-value` = ifelse(is.na(`P-value`), "Ref", `P-value`)
      )

counts %>%
  arrange(match(group, ref_level)) %>%
  select(all_of(c("Variable (Level)", "Cases", "Controls", "Total (n, %)", "HR (95% CI)", "P-value")))


  # This fixes the select issue
  return(counts %>% dplyr::select(`Variable (Level)`, Cases, Controls, `Total (%)`, HR (95% CI), P-value))
})

In [55]:
temp_cat_demo <- c("Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "Region", "BMI_GROUP_impute", "PanLocation", "OthMetFLBase")


In [61]:
DemographicOth_filtered <- DemographicOth[!DemographicOth %in% c("AgeAtDiagnosis", "BMI_imputed", "BMI")]

all_vars <- c(DemographicOth_filtered, comorbidity, Exposure1, Exposure2)

dataset <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

summary_cat <- map_dfr(all_vars, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

if (var != "AgeAtDiagnosis" & var != "BMI_imputed") {
  counts <- df %>%
    group_by(group = .data[[var]], indicator) %>%
    summarise(n = n(), .groups = "drop") %>%
    pivot_wider(names_from = indicator, values_from = n, values_fill = 0, names_prefix = "Group_") %>%
    mutate(
      group = as.character(group),
      Total = Group_0 + Group_1,
      Pct = round(Total / sum(Total) * 100, 1),
      `Variable` = var,
      `Level` = group,
      `Cases` = Group_1,
      `Controls` = Group_0,
      `Total (%)` = paste0(Total, " (", Pct, "%)"),
       Var_to_merge = paste0(var,group)
    )

  # This fixes the select issue
  return(counts %>% dplyr::select(`Variable`, Level,Cases, Controls, `Total (%)`))
}
})

display_df(summary_cat,100)

In [60]:
DemographicOth_filtered <- DemographicOth[!DemographicOth %in% c("AgeAtDiagnosis", "BMI_imputed", "BMI")]

all_vars <- c(DemographicOth_filtered, comorbidity, Exposure1, Exposure2)

dataset <- cases_control_allvar_new %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

summary_cat <- map_dfr(all_vars, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

if (var != "AgeAtDiagnosis" & var != "BMI_imputed") {
  counts <- df %>%
    group_by(group = .data[[var]], indicator) %>%
    summarise(n = n(), .groups = "drop") %>%
    pivot_wider(names_from = indicator, values_from = n, values_fill = 0, names_prefix = "Group_") %>%
    mutate(
      group = as.character(group),
      Total = Group_0 + Group_1,
      Pct = round(Total / sum(Total) * 100, 1),
      `Variable` = var,
      `Level` = group,
      `Cases` = Group_1,
      `Controls` = Group_0,
      `Total (%)` = paste0(Total, " (", Pct, "%)"),
       Var_to_merge = paste0(var,group)
    )

  # This fixes the select issue
  return(counts %>% dplyr::select(`Variable`, Level,Cases, Controls, `Total (%)`))
}
})

display_df(summary_cat,100)

In [69]:
results <- lapply(temp_cat_demo, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))
  if (n_distinct(df[[var]]) < 2) return(NULL)
  
  model <- coxph(as.formula(paste("Surv(AVAL, indicator) ~", var)), data = df)
  s <- summary(model)
  
  hr_df <- data.frame(
    Var_to_merge = rownames(s$coef),
    HR = round(s$coef[, 2], 2),
    LCI = round(s$conf.int[,"lower .95"], 2),
    UCI = round(s$conf.int[,"upper .95"], 2),
    Pval = s$coef[, 5]
  )
  
  return(hr_df)
})

results_df <- bind_rows(results)
display(results_df)


In [22]:
library(dplyr)
library(gtsummary)
library(survival)
library(car)
library(parameters)

# Combine all variables you want
all_vars <- c(
  comorbidity,
  Exposure1,
  Exposure2,
  DemographicOth
)

In [23]:
# Add survival outcome variables
all_vars_full <- c("Sex", "Age_Group","exposure", "Race", "CKD", "CLD","indicator", "AVAL")

# Univariable Cox regression table
uv_tb <- cases_control_allvar_new %>%
  select(all_of(all_vars_full)) %>%
  tbl_uvregression(
    method = coxph,
    y = Surv(AVAL, indicator),
    exponentiate = TRUE
  ) %>%
  add_global_p()

gt(uv_tb) %>% print()

In [66]:
# Add survival outcome variables
all_vars_full <- c("Sex", "Age_Group","exposure", "Race", "CKD", "CLD","indicator", "AVAL")

# Univariable Cox regression table
uv_tb <- cases_control_allvar_new %>%
  select(all_of(all_vars_full)) %>%
  tbl_uvregression(
    method = coxph,
    y = Surv(AVAL, indicator),
    exponentiate = TRUE
  ) %>%
  add_global_p()

table1_formatted <- uv_tb %>% gt() 
print(table1_formatted)

In [28]:
tbl <- cases_control_allvar_new %>%
  select(all_of(all_vars_full)) %>%
  tbl_summary()  

In [ ]:
#Option B: Convert to dataframe for SparkR-compatible output

uv_df <- as.data.frame(uv_tbl$table_body)
head(uv_df)

createDataFrame(uv_df) %>% showDF()

write.csv(uv_df, "cox_univariable_summary.csv", row.names = FALSE)

In [ ]:
# Table 2: Lab Variables
lab_vars <- lab
summary_lab <- map_dfr(lab_vars, function(var) {
  df <- dataset %>% filter(!is.na(.data[[var]]))

  total_n <- nrow(filter_data)
  non_missing <- nrow(df)
  missing_pct <- paste0(round((1 - non_missing / total_n) * 100, 1), "%")
  median_case <- round(median(df[[var]][df$indicator == 1], na.rm = TRUE), 1)
  median_control <- round(median(df[[var]][df$indicator == 0], na.rm = TRUE), 1)
  IQR_case <- round(IQR(df[[var]][df$indicator == 1], na.rm = TRUE),1)
  IQR_control <- round(IQR(df[[var]][df$indicator == 0], na.rm = TRUE),1)

  model <- tryCatch(coxph(Surv(AVAL, indicator) ~ .data[[var]], data = df), error = function(e) NULL)
  if (!is.null(model)) {
    s <- summary(model)
    HR <- round(s$coef[2], 2)
    LCI <- round(s$conf.int[,"lower .95"], 2)
    UCI <- round(s$conf.int[,"upper .95"], 2)
    p_val <- format_pval(s$coef[5])
  }

  return(data.frame(
    Variable = var,
    `Missing %` = missing_pct,
    `Cases (Median, IQR)` = paste0(median_case," (",IQR_case,")"),
    `Controls (Median, IQR)` = paste0(median_control," (",IQR_control,")"),
    `HR (95% CI)` = paste0(HR, " (", LCI, ", ", UCI, ")"),
    `P-value` = p_val
  ))
})

#### Format Tables

In [ ]:
# Format Table 1
table1_formatted <- summary_cat %>%
  gt() %>%
  tab_header(title = "Table 1: Baseline Characteristics and Univariate Cox Results")

# Format Table 2
table2_formatted <- summary_lab %>%
  gt() %>%
  tab_header(title = "Table 2: Lab Characteristics with Summary and Cox Results")

# Output Tables
table1_formatted
table2_formatted


In [36]:
nrow(df_lab)

In [17]:
coxph(Surv(AVAL,indicator)~Sex, data = filter_data)

In [ ]:
par(2,2)

# Plot cross-validated partial likelihood
display_plot(plot(lasso_fit))

# Coefficients at best lambda (lambda.1se is more conservative)
coef_lasso <- coef(lasso_fit, s = "lambda.1se")
active_vars_lasso <- rownames(coef_lasso)[coef_lasso != 0]

# Show selected variables
print(active_vars_lasso)

# Plot cross-validation curve
display_plot(plot(ridge_fit))

# Coefficients at lambda.min
coef_ridge <- coef(ridge_fit, s = "lambda.1se")

# View all coefficients (none will be exactly zero)
print(coef_ridge)


In [ ]:
#Stepwise Selection (Direction Both) - p value based - Whole Data
stepwise_both <- fastbw(cox_full, rule = "p", slentry = 0.15, slstay = 0.20, direction = "both")

#Stepwise Selection (Direction Both) - p value based - Whole Data
stepwise_backwrd <- fastbw(cox_full, rule = "p", slentry = 0.15, slstay = 0.20, direction = "backward")

#Stepwise Selection (Direction Backward) - AIC based - Remove baseline LM

#Stepwise Selection (Direction Backward) - p value based - Remove baseline LM


In [43]:
# correlation between medication and comorbidities

#library(corrplot)

lab_cols <- c("ALT", "AST", "Albumin", "ALP", "TotalBilirubin", "DirectBilirubin", "TotalProtein", "ProthrombinT", "SerumLipase", "SerumPanAmylase")

# Subset
lab_cor<- lab_test_result[ , lab_cols]

# Pairwise complete correlation matrix
corr_matrix <- cor(lab_cor, use = "pairwise", method = "spearman")

cordata <- as.data.frame(corr_matrix) %>% round(.,2)
# Visualize
#corrplot(corr_matrix, method = "color", tl.col = "black", tl.cex = 0.8)


In [51]:
library(dplyr)

# Variables to analyze
categorical_vars <- c("Age_Group", "Ethnicity", "Race", "Sex", "MaritalStatus", "BMI_Group", "BMI","OthMetFLBase", "PanLocation", 
"CKD", "T2DM", "Hepatitis", "Hypertension", "CLD", "Hyperlipidemia", "osa", "COPD", "Anxiety", "Ischemic_Heart_Disease", "Depression", "Obesity_codes","Cancer",
"Gastroesophageal_refluxdisease", "abdominal_pain","Dyspnea","Anemia","FHOMND","comorbidScore", "RadiationFl1", "SurgeryFl1","AdvChemoMedFl1")

continuous_vars <- "AgeAtDiagnosis"  # replace with your real continuous variables

data <- cases_control_allvar %>% filter(LivDtFl30 == "Y")

# =====================
# Handle Categorical Vars
# =====================
cat_results <- lapply(categorical_vars, function(var) {
  # Recode "Unknown"/NA
  data_labeled <- data %>%
    mutate(.var = .data[[var]],
           .var = ifelse(is.na(.var) | .var == "Unknown", "Unknown/NA", as.character(.var)))

  # Summary
  summary_table <- data_labeled %>%
    group_by(group = .var) %>%
    summarise(
      n = n(),
      median_time = median(AVAL, na.rm = TRUE),
      iqr = IQR(AVAL, na.rm = TRUE),
      .groups = "drop"
    )

  # Exclude unknown for test
  test_data <- data_labeled %>% filter(.var != "Unknown/NA")
  num_levels <- n_distinct(test_data$.var)

  if (num_levels == 2) {
    test <- wilcox.test(AVAL ~ .var, data = test_data)
    p_value <- round(test$p.value, 4)
    test_type <- "Wilcoxon"
  } else if (num_levels > 2) {
    test <- kruskal.test(AVAL ~ .var, data = test_data)
    p_value <- round(test$p.value, 4)
    test_type <- "Kruskal-Wallis"
  } else {
    return(list(variable = var, message = "Skipped (fewer than 2 levels)"))
  }

  # Add metadata
  summary_table <- summary_table %>%
    mutate(
      p_value = ifelse(group == "Unknown/NA", NA, p_value),
      variable = var,
      test_type = test_type
    )

  list(
    variable = var,
    summary_table = summary_table,
    test_result = test
  )
})

# =====================
# Handle Continuous Vars
# =====================
cont_results <- lapply(continuous_vars, function(var) {
  # Drop NA
  df_var <- data %>% filter(!is.na(.data[[var]]), !is.na(AVAL))

  # Correlation test
  test <- cor.test(df_var[[var]], df_var$AVAL, method = "spearman")
  cor_val <- round(test$estimate, 3)
  p_val <- round(test$p.value, 4)

  # Optional: split into high/low groups for summary
  median_cut <- median(df_var[[var]], na.rm = TRUE)
  df_var <- df_var %>%
    mutate(group = ifelse(.data[[var]] <= median_cut, "Low", "High"))

  summary_table <- df_var %>%
    group_by(group) %>%
    summarise(
      n = n(),
      median_time = median(AVAL, na.rm = TRUE),
      iqr = IQR(AVAL, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    mutate(
      variable = var,
      test_type = "Spearman Correlation",
      p_value = p_val,
      spearman_rho = cor_val
    )

  list(
    variable = var,
    summary_table = summary_table,
    test_result = test
  )
})

# =====================
# Combine & View Output
# =====================
all_tables <- do.call(bind_rows, c(
  lapply(cat_results, function(x) x$summary_table),
  lapply(cont_results, function(x) x$summary_table)
))

print(all_tables)

# Optional: write to CSV
# write.csv(all_tables, "time_to_metastasis_summary.csv", row.names = FALSE)


In [44]:
cordata

In [47]:
colnames(dataset)

In [47]:
# Remove subjects with UNK sex
#df1 <- cases_control_allvar[!(cases_control_allvar$Sex == "Unknown"),]
sum(cases_control_allvar$MaritalStatus== "Unknown")
#nrow(df1)

In [30]:
df2 <- cases_control_allvar %>%
  mutate(exposure = case_when(
    AdvChemoMedFl1 == "Y" ~ 1,
    SurgeryFl1 == 1 ~ 2,
    RadiationFl1 == 1 ~ 3,
    TRUE ~ 0
    ),
    BMI_Group = ifelse(is.na(BMI), "UNK", BMI_Group)
  )


In [46]:
#Run Univariate for BMI cont and BMI group with BMI = UNK value 
coxph(Surv(AVAL,indicator) ~ BMI, data = df2)
              
#

In [42]:
# Remove chemo, surg, radio, martial status add expo and run reg model again
filter_data <- df2 %>% filter(LivDtFl == "N" | LivDtFl30 == "Y")

df3 <- df1[!(df1$Race == "Unknown" | df1$Ethnicity == "Unknown"),]

dataset <- df1
intercept_only <- coxph(Surv(AVAL,indicator)~1,data = dataset)

# Add the Exposure related variables
cox_full2 <- coxph(Surv(AVAL,indicator)~ AgeAtDiagnosis + Ethnicity  + Sex + Race + OthMetFLBase + PanLocation +
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD + Anxiety + Ischemic_Heart_Disease + Depression + Obesity_codes + Cancer +
Gastroesophageal_refluxdisease + abdominal_pain + Dyspnea + Anemia + FHOMND, data = dataset)

step_both_model <- step(intercept_only, direction='both', scope=formula(cox_full2), trace=0)

summary(step_both_model)

In [65]:
library(boot)
library(tableone)
library(table1)
library(survival)

table1(~ AgeAtDiagnosis + Ethnicity  + Sex + Race + MaritalStatus + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD | indicator, data=cases_control_allvar_new)

In [63]:
library(table1)

pvalue <- function(x, ...) {
  # Construct vectors of data y, and groups (strata) g
  y <- unlist(x)
  g <- factor(rep(1:length(x), times=sapply(x, length)))
  if (is.numeric(y)) {
    # For numeric variables, perform a standard 2-sample t-test
    p <- t.test(y ~ g)$p.value
  } else {
    # For categorical variables, perform a chi-squared test of independence
    p <- chisq.test(table(y, g))$p.value
  }
  # Format the p-value, using an HTML entity for the less-than sign.
  # The initial empty string places the output on the line below the variable label.
  c("", sub("<", "&lt;", format.pval(p, digits=3, eps=0.001)))
}

table1(~ AgeAtDiagnosis + Ethnicity  + Sex + Race + MaritalStatus + OthMetFLBase + PanLocation + 
CKD + T2DM + Hepatitis + CLD + Hypertension + osa +  Hyperlipidemia + COPD  | indicator,
       data=cases_control_allvar_new,overall=F, extra.col=list(`P-value`=pvalue))
print_help